# Build a Q&A application with Bedrock, Llama Index and Langchain

This notebook explains steps requried to build a Question & Answer application with Bedrock - Anthropic Claude model, Langchain and LLama index

(This notebook was tested on SageMaker Studio ml.m5.2xlarge instance with Datascience 3.0 kernel)

## Pre-requisites

In [21]:
!pip install langchain --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 237.3 kB/s eta 0:00:00:0100:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.245
    Uninstalling langchain-0.0.245:
      Successfully uninstalled langchain-0.0.245

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [22]:
!pip install llama-index

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 6.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 23.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: fsspec
    Found existing

In [23]:
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install sagemaker --upgrade

NOTE: This notebook requires Bedrock Python SDK. Install Bedrock SDK if you haven't done yet. Refer to 00_bedrock_onboarding.ipynb notebook for steps to install and uninstall previous version if any.

## Restart Kernel

In [ ]:
#Restart Kernel after the installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

## Setup depedencies

In [24]:
#Check Python version is greater than 3.8 which is required by Langchain if you want to use Langchain
import sys
sys.version

'3.10.6 (main, Oct  7 2022, 20:19:58) [GCC 11.2.0]'

In [25]:
assert sys.version_info >= (3, 8)

In [50]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [41]:
from llama_index import download_loader, SimpleDirectoryReader
from llama_index import  LangchainEmbedding, VectorStoreIndex
from llama_index import LLMPredictor, ServiceContext
#import torch

import langchain
#from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [35]:
import os, json
from tqdm import tqdm
import pathlib
from pathlib import Path

In [29]:
import boto3
import sagemaker
session = boto3.Session()
sagemaker_session = sagemaker.Session()
studio_region = sagemaker_session.boto_region_name 
bedrock = session.client("bedrock", region_name=studio_region)

In [38]:
# Put your directory containing PDFs here
pdf_file_name = 'Amazon_Fire_TV_User_Guide.pdf'
pdf_path = f'pdfs/firetv/{pdf_file_name}'

In [36]:
index_directory= f'llama_indices'
index_path = f'{index_directory}/{pdf_file_name}'
pathlib.Path(index_directory).mkdir(parents=True, exist_ok=True)

## Load data with PDFReader

In [39]:
pdf_loader = download_loader('PDFReader')
loader = pdf_loader()
docs = loader.load_data(file=Path(pdf_path))

## Create Huggingface Embeddings and Embeddings model

In [47]:
from langchain.embeddings import HuggingFaceEmbeddings
emb = HuggingFaceEmbeddings()
emb_model = LangchainEmbedding(emb)

In [45]:
from langchain.llms.bedrock import Bedrock

#Creating Anthropic Claude
model_args= {'max_tokens_to_sample':200,'temperature':0}
llm = Bedrock(model_id="anthropic.claude-v1", client=bedrock, model_kwargs=model_args)
llm_predictor = LLMPredictor(llm=llm)

In [48]:
sc = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=emb_model)

## Persist with Vector store

In [49]:
index = VectorStoreIndex.from_documents(docs,service_context=sc)
index.storage_context.persist(persist_dir=index_path)

## Query LLM with Index

In [55]:
from llama_index import StorageContext, load_index_from_storage
stc = StorageContext.from_defaults(persist_dir=index_path)
index = load_index_from_storage(stc, service_context=sc)
engine = index.as_query_engine()

In [87]:
query = "How to setup Parental controls?"
resp = engine.query(query)

In [88]:
#Print Query Response
print(resp.response)


To set up Parental Controls on an Amazon Fire TV device:

1. From the Home screen, select Settings, and then select Parental Controls.
2. Using your remote, press the Select button to turn parental controls On. 
3. Enter your Parental Controls PIN, and then select Next. Your PIN is the same PIN you use for other Amazon services such as Amazon Instant Video.
4. After you set a PIN, you can set one or more of the following settings:

- Require a PIN for all purchases: Blocks the ability to make any purchases without entering the PIN.
- Require a PIN for Amazon Instant Video only: Only requires a PIN to access Amazon Instant Video content. 
- Block the ability to view or purchase certain content types, such as games, apps, or photos.
- Change your Parental Controls PIN: Allows you to set a new PIN for Parent


In [75]:
#Browse through the source nodes
print(f'Total number of source nodes {len(resp.source_nodes)}')

Total number of source nodes 2


In [85]:
for i, n in enumerate(resp.source_nodes):
    print(f'Node {i +1}') 
    print('-' * 20)    
    print(f'text : {n.node.text} \n')
    print(f'metadata : {n.node.metadata}\n')


Node 1
--------------------
text : Set Up Parental Controls
With Parental Controls, you can block purchases and restrict access to Amazon movies, TV shows,
games, apps, photos, and more.
Note: Parental controls do not restrict content in third-party applications. Parental controls for third-party applications
are determined by the app provider.
When entering the PIN, you will need to use an Amazon remote or the Fire TV Remote App. You cannot
use a third-party remote.
1. From the Home  screen, select Settings , and then select Parental Controls .
2. Using your remote, press the Select  button to turn parental controls On or Off.
3. Enter your Parental Controls PIN, and then select Next .
Your PIN is the same PIN you use for other Amazon services such as Amazon Instant Video.
Note: If you forget your PIN, go to Amazon Instant Video SettingsAmazon Instant Video Settings  (Amazon.com full
site)Amazon Instant Video Settings (https://www.amazon.com/video/settings) to reset your Parental Cont